1. Import data.

In [74]:
import pandas as pd

df = pd.read_csv('C:/Users/TheAncientOwl/Code/data-analysis-tool/server/Date.csv')

df.head()


,Country,EA,EAM,EAF,AFF,ANE,AME,CY,SA,CP,LA,LCP,ARL,AGL
0,Aruba,24.125617,25.060201,22.962485,10.503020,13471.23596,20695.5618,3656.505885,180.0,17824090.07,180.0,4600273.024,2000.0,20.00
1,Afghanistan,43.130001,37.259998,65.769997,22.042897,4300.00000,10450.0000,2164.900000,652860.0,4140568.00,652860.0,1912634.000,7703000.0,379190.00
2,Albania,37.290001,33.720001,42.279999,18.440931,960.00000,2010.0000,4840.500000,28750.0,678196.00,27400.0,140110.000,611346.0,11740.81
3,Andorra,24.125617,25.060201,22.962485,10.503020,0.00000,0.0000,3656.505885,470.0,17824090.07,470.0,4600273.024,830.0,188.30
4,Armenia,25.780001,23.400000,28.650000,13.942312,1070.00000,680.0000,2623.600000,29740.0,334342.00,28470.0,127436.000,445600.0,16773.00


2. Pick features & target.

In [75]:
target = 'Country'
features = list(df.columns)
features.remove(target)

print(f">> Target: '{target}'")
print(f'>> Features: {features}')


>> Target: 'Country'
>> Features: ['EA', 'EAM', 'EAF', 'AFF', 'ANE', 'AME', 'CY', 'SA', 'CP', 'LA', 'LCP', 'ARL', 'AGL']


3. Standardize the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

x = df.loc[:, features].values
y = df.loc[:, target].values

x = StandardScaler().fit_transform(x)
features_df = pd.DataFrame(data=x, columns=features)



4. Correlation matrix.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = features_df.corr().round(2)

plt.figure(figsize=(8, 8))
cmap = sns.diverging_palette(250, 20, as_cmap=True)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, annot=True, 
            vmax=1, vmin=-1, center=0,
            cmap=cmap, mask=mask)
plt.show()


5. PCA.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

principal_components = pca.fit_transform(x)

labels = [f'PC{x}' for x in range(1, pca.n_components_ + 1)]
principal_df = pd.DataFrame(data=principal_components, columns=labels)
final_df = pd.concat([df[target], principal_df], axis=1)

final_df.head()

6. Pick principal components count.

6.1. *via* Kaiser.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pc_values = np.arange(pca.n_components_) + 1
plt.plot(pc_values, pca.explained_variance_ratio_, 'ro-', linewidth=2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.xticks(range(1, pca.n_components_ + 1))
plt.show()

6.2. *via* The threshold of variance explained approach *(at least 70%)*.

In [ ]:
import numpy as np

explained_variance_ratio = pca.explained_variance_ratio_ * 100
out_sum = np.cumsum(explained_variance_ratio)

pcs = [f'PC{x}' for x in range(1, pca.n_components_ + 1)]
variance_ratio_df = pd.DataFrame(data={'Proportion of Variance Explained': list(explained_variance_ratio), 'Cumulative Proportion of Variance Explained': list(out_sum)}, 
                           columns=['Proportion of Variance Explained', 'Cumulative Proportion of Variance Explained'],
                           index=pcs)
variance_ratio_df

6.3. *via* Hold onto principal components whose eigenvalues are > 1.

In [ ]:
variance_ratio_df = pd.DataFrame(data={'Explained Variance': list(pca.explained_variance_)}, 
                                 columns=['Explained Variance'],
                                 index=pcs)
variance_ratio_df

7. Interpret final PCA solution.

7.1. Execute PCA with chosen principal components count

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)

principal_components = pca.fit_transform(x)

labels = [f'PC{x}' for x in range(1, pca.n_components_ + 1)]
principal_df = pd.DataFrame(data=principal_components, columns=labels)
final_df = pd.concat([df[target], principal_df], axis=1)

final_df.head()


7.2. Loadings.

In [ ]:
loadings = pd.DataFrame(pca.components_.T.round(3), columns=labels, index=features)
loadings


In [ ]:
# pca general form
for pca_id in range(1, pca.n_components_ + 1):
  print(f'W{pca_id} = ', sep='', end='')
  for feature, row in loadings.iterrows():
    print(f'({row[pca_id - 1]} * {feature})', end=' + ')
  print('\n')

7.3. Loadings Matrix.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

loadings_matrix = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_), columns=labels, index=features)

plt.figure(figsize=(8, 8))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(loadings_matrix, annot=True, 
            vmax=1, vmin=-1, center=0,
            cmap=cmap)
plt.show()


8. PCs visualize 2D Projection

In [ ]:
from matplotlib import pyplot as plt
import random

#? @param pc_x: label of x component f'PC{index}'
#? @param pc_y: label of y component f'PC{index}'
#? @param targets: targets to be scattered
#? @param annot: if the scattered plots should have titles
#? @param legend: if the plot should have legend 
def plot_principal_components(pc_x, pc_y, targets, annot=False, legend=False):
  fig = plt.figure(figsize=(8, 8))
  
  ax = fig.add_subplot(1, 1, 1)
  ax.set_xlabel(pc_x, fontsize=15)
  ax.set_ylabel(pc_y, fontsize=15)
  ax.set_title(f'{pc_x} & {pc_y}', fontsize=20)

  for obs in targets:
    indices_to_keep = df[target] == obs

    x_points = final_df.loc[indices_to_keep, pc_x]
    y_points = final_df.loc[indices_to_keep, pc_y]

    ax.scatter(x_points, y_points, s=50)
    if annot:
      for x, y in zip(x_points, y_points):
        ax.annotate(obs, (x, y))

  if legend:
    ax.legend(targets)

  ax.grid()
  plt.show()

# targets = set(df[target])
random.seed(893927)
targets = set(random.sample(sorted(df[target].values), 10))
plot_principal_components('PC1', 'PC2', targets, annot=True)